In [1]:
!pip install psycopg2

In [2]:
import psycopg2

# データベース接続情報
host = 'postgis'
port = '5432'
user = 'postgres'
password = 'postgres'
database = 'sandbox'

# PostGISへの接続
connection = psycopg2.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database
)

# クエリの実行
cursor = connection.cursor()

# getTableAndColumnsName
schema = "public"
query = f"""
SELECT
  t.table_name,
  c.*
FROM
  information_schema.tables t
    JOIN information_schema.columns c
      ON t.table_name = c.table_name
WHERE
  t.table_schema = '{schema}'
    AND c.table_schema = '{schema}'
ORDER BY
  t.table_name,
  c.ordinal_position;
"""

# クエリの実行と結果の取得
cursor.execute(query)
results = cursor.fetchall()

# 結果の表示
table_info = []
for row in results:
    table_name = row[3]
    if table_name == "geography_columns":
        continue
    if table_name == "geometry_columns":
        continue
    if table_name == "spatial_ref_sys":
        continue
    column_name = row[4]
    nullable = row[7]
    data_type = row[8]
    data_type_sub = row[28]
    table_info_row = f"table_name: {table_name}, column_name: {column_name}, data_type: {data_type}/{data_type_sub}, nullable: {nullable}"
    table_info.append(table_info_row)

# コネクションのクローズ
cursor.close()
connection.close()

print("\n".join(table_info))

table_name: flood_areas_akita_2023_07_18, column_name: gridcode, data_type: bigint/int8, nullable: YES
table_name: flood_areas_akita_2023_07_18, column_name: flood_depth, data_type: text/text, nullable: YES
table_name: flood_areas_akita_2023_07_18, column_name: geometry, data_type: USER-DEFINED/geometry, nullable: YES
table_name: flood_areas_akita_2023_07_19, column_name: geometry, data_type: USER-DEFINED/geometry, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: id, data_type: bigint/int8, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: name, data_type: character varying/varchar, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: name_en, data_type: character varying/varchar, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: geom, data_type: USER-DEFINED/geometry, nullable: YES
table_name: parks, column_name: id, data_type: bigint/int8, nullable: YES
table_name: parks, column_name: name, data_type: character varying

In [ ]:
import os

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
print(len(OPENAI_API_KEY))


In [20]:
from langchain import OpenAI, PromptTemplate

llm = OpenAI(model_name="text-davinci-003", temperature=0)

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"

Only use the following tables:

{table_info}

Examples:
Question: 秋田県の浸水エリア内にある公園を教えて
SQLQuery: SELECT p.id, p.name, p.name_en FROM parks_akita_2023_07_18 AS p WHERE EXISTS (SELECT 1 FROM flood_areas_akita_2023_07_18 AS f WHERE ST_Intersects(p.geom, f.geometry));
Note: do not forget `SELECT 1` in subquery

Question: {input}"""

prompt_template = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

prompt = prompt_template.format(dialect="PostGIS SQL", table_info="\n".join(table_info), input="秋田県の浸水エリア内にある学校を教えて", )

print(prompt)

sql_query = llm(prompt).replace('SQLQuery: ', '').split("\n")[1]
print(sql_query)


Given an input question, first create a syntactically correct PostGIS SQL query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"

Only use the following tables:

table_name: flood_areas_akita_2023_07_18, column_name: gridcode, data_type: bigint/int8, nullable: YES
table_name: flood_areas_akita_2023_07_18, column_name: flood_depth, data_type: text/text, nullable: YES
table_name: flood_areas_akita_2023_07_18, column_name: geometry, data_type: USER-DEFINED/geometry, nullable: YES
table_name: flood_areas_akita_2023_07_19, column_name: geometry, data_type: USER-DEFINED/geometry, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: id, data_type: bigint/int8, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: name, data_type: character varying/varchar, nullable: YES
table_name: hospitals_akita_2023_07_18, column_name: name_en, data_type: character varying/varcha

In [21]:
# PostGISへの接続
connection = psycopg2.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database
)

# クエリの実行
cursor = connection.cursor()

# クエリの実行と結果の取得
cursor.execute(sql_query)
results = cursor.fetchall()

# 結果の表示
print("結果:", results)

# コネクションのクローズ
cursor.close()
connection.close()

結果: [(387752594, '秋田市立城東中学校', ''), (387752600, '秋田市立東小学校', '')]
